In [2]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [315]:
import pandas as pd
import numpy as np
import pdfplumber
import tabula
from Olympic_PED_use.src import functions as fn

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from bs4 import BeautifulSoup
import certifi
import urllib3
import re
from csv import DictReader, DictWriter

In [4]:
from time import sleep
from random import randint
import requests
from requests import get

In [39]:
import pandas as pd
pd.set_option('display.max_rows', 2600)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000000)

#### Function to scrape and return results as a dataframe

In [6]:
def wiki_scraper(content):
    a = []
    b = []
    c = []
    d = []
    e = []
    f = []
    g = []
    tables = []
    for table in content:
        tables.append(table)
        for table_ in tables:
            rows = table_.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if len(cells) == 7:
                    a.append(cells[0].text)
                    b.append(cells[1].text)
                    c.append(cells[2].text)
                    d.append(cells[3].text)
                    e.append(cells[4].text)
                    f.append(cells[5].text)
                    g.append(cells[6].text)
    df = pd.DataFrame(a, columns=['name'])
    df['country'] = b
    df['event'] = c
    df['date_of_violation'] = d
    df['substance'] = e
    df['sanction'] = f
    df['references'] = g
    
    return df

#### Retrieving contents of doping cases in athletics Wikipedia page

In [7]:
url = 'https://en.wikipedia.org/wiki/List_of_doping_cases_in_athletics'
req = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',
                         ca_certs=certifi.where())
res = req.request('GET', url)
soup = BeautifulSoup(res.data, 'html.parser')
contents = soup.find_all('table', class_='wikitable sortable')   

#### Formatting the values in the dataframe by removing '\n' 

In [8]:
def col_format(df):
    df['name'] = [x.strip('\n') for x in df['name']]
    df['country'] = [x.strip('\n') for x in df['country'].values]
    df['date_of_violation'] = [x.strip('\n') for x in df['date_of_violation'].values]
    df['event'] = [x.strip('\n') for x in df['event'].values]
    df['substance'] = [x.strip('\n') for x in df['substance'].values]
    df['sanction'] = [x.strip('\n') for x in df['sanction'].values]
    df['references'] = [x.strip('\n') for x in df['references'].values]
    return df

#### Creating doping dataframe and adding a 'flagged' column specifying whether or not the athlete has tested positive for PED use during the Games.

In [9]:
doping = wiki_scraper(contents)

doping = col_format(doping)

doping['flagged'] = [1 for x in doping.index]

#### Reading in the Athlete events csv file containing Olympic athletes and general information

In [10]:
athlete_df = pd.read_csv('../data/athlete_events.csv')
athlete_df.columns = [x.lower() for x in athlete_df.columns]

#### Merging the athlete and doping dataframes by name

In [11]:
df = pd.merge(athlete_df, doping, on='name', how='left')
df = df.drop(['sanction', 'references', 'country', 'date_of_violation', 'games'], axis=1)
df = df[df['sport']=='Athletics']
df = df[df['year'] > 2003]
df = df[df['year'] < 2017]
df

,id,name,sex,age,height,weight,team,noc,year,season,city,sport,event_x,medal,event_y,substance,flagged
98,34,Jamale (Djamel-) Aarrass (Ahrass-),M,30.0,187.0,76.0,France,FRA,2012,Summer,London,Athletics,"Athletics Men's 1,500 metres",NaN,NaN,NaN,NaN
135,49,Moonika Aava,F,24.0,168.0,65.0,Estonia,EST,2004,Summer,Athina,Athletics,Athletics Women's Javelin Throw,NaN,NaN,NaN,NaN
136,49,Moonika Aava,F,28.0,168.0,65.0,Estonia,EST,2008,Summer,Beijing,Athletics,Athletics Women's Javelin Throw,NaN,NaN,NaN,NaN
148,55,Antonio Abadia Beci,M,26.0,170.0,65.0,Spain,ESP,2016,Summer,Rio de Janeiro,Athletics,"Athletics Men's 5,000 metres",NaN,NaN,NaN,NaN
163,67,Mariya Vasilyevna Abakumova (-Tarabina),F,22.0,179.0,80.0,Russia,RUS,2008,Summer,Beijing,Athletics,Athletics Women's Javelin Throw,Silver,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275330,135547,Viktoriya Viktorovna Zyabkina,F,19.0,174.0,62.0,Kazakhstan,KAZ,2012,Summer,London,Athletics,Athletics Women's 200 metres,NaN,NaN,NaN,NaN
275331,135547,Viktoriya Viktorovna Zyabkina,F,23.0,174.0,62.0,Kazakhstan,KAZ,2016,Summer,Rio de Janeiro,Athletics,Athletics Women's 100 metres,NaN,NaN,NaN,NaN
275332,135547,Viktoriya Viktorovna Zyabkina,F,23.0,174.0,62.0,Kazakhstan,KAZ,2016,Summer,Rio de Janeiro,Athletics,Athletics Women's 200 metres,NaN,NaN,NaN,NaN
275333,135547,Viktoriya Viktorovna Zyabkina,F,23.0,174.0,62.0,Kazakhstan,KAZ,2016,Summer,Rio de Janeiro,Athletics,Athletics Women's 4 x 100 metres Relay,NaN,NaN,NaN,NaN


#### Creating dataframe for the 2016 Games held in Athens

In [685]:
df_16 = df[df['year']==2016]
df_16 = df_16.drop_duplicates().reset_index(drop=True)
df_16.name = df_16.name.str.replace(r"\(.*\)","")


#### Next, I am going to scrape the tables from Olympic.org that contain the different events, athletes per event, ranking, and result (time/distance). I am going to start with the 100m Dash

In [686]:
url = 'https://www.olympic.org/rio-2016/athletics/100m-men'
req = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',
                         ca_certs=certifi.where())
res = req.request('GET', url)
soup = BeautifulSoup(res.data, 'html.parser')
contents = soup.find_all(class_='table4')  

In [687]:
mens_100m_16 = fn.athens_scraper(contents)
mens_100m_16 = fn.content_cleaner(mens_100m_16)
mens_100m_16.columns = ['rank', 'name', 'result']
mens_100m_16.insert(3, "event_x", "Athletics Men's 100 metres")
mens_100m_16

,rank,name,result,event_x
0,1.,Usain Bolt,9.81,Athletics Men's 100 metres
1,2.,Justin Gatlin,9.89,Athletics Men's 100 metres
2,3.,Andre De Grasse,9.91,Athletics Men's 100 metres
3,4.,Yohan Blake,9.93,Athletics Men's 100 metres
4,5.,Akani Simbine,9.94,Athletics Men's 100 metres
5,6.,Ben Youssef Meite,9.96,Athletics Men's 100 metres
6,7.,Jimmy Vicaut,10.04,Athletics Men's 100 metres
7,8.,Trayvon Bromell,10.06,Athletics Men's 100 metres


In [688]:
mens_100m_16.name = [x.strip() for x in mens_100m_16.name]

In [689]:
mens_100m_16

,rank,name,result,event_x
0,1.,Usain Bolt,9.81,Athletics Men's 100 metres
1,2.,Justin Gatlin,9.89,Athletics Men's 100 metres
2,3.,Andre De Grasse,9.91,Athletics Men's 100 metres
3,4.,Yohan Blake,9.93,Athletics Men's 100 metres
4,5.,Akani Simbine,9.94,Athletics Men's 100 metres
5,6.,Ben Youssef Meite,9.96,Athletics Men's 100 metres
6,7.,Jimmy Vicaut,10.04,Athletics Men's 100 metres
7,8.,Trayvon Bromell,10.06,Athletics Men's 100 metres


#### This only matched 3 of the 7 values in the result column. There are names in the athletes df that include first, middle, and last names. I am going to transform the names in the dataframe to only be first and last names. First, I will create a list to hold the first and last names, then replace the names in the dataframe with the names from the list.

In [690]:
names = []
for x in df_16.name:
    split = x.split()
    x = split[0] + ' ' + split[-1]
    names.append(x)


df_16['name'] = [x for x in names]

In [691]:
names = []
for x in mens_100m_16.name:
    split = x.split()
    x = split[0] + ' ' + split[-1]
    names.append(x)


mens_100m_16['name'] = [x for x in names]

In [692]:
def format_name(df):
    df.name = [x.strip() for x in df.name]
    names = []
    for x in df.name:
        split = x.split()
        x = split[0] + ' ' + split[-1]
        names.append(x)
    df.name = [x for x in names]
    return df

In [693]:
df_16 = pd.merge(df_16, mens_100m_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_100m_mens'))
df_16.result.value_counts()

9.89     1
10.06    1
9.93     1
9.81     1
9.94     1
9.91     1
10.04    1
Name: result, dtype: int64

In [694]:
def olympic_query(event):
    url = 'https://www.olympic.org/rio-2016/athletics/' + str(event)
    req = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',
                             ca_certs=certifi.where())
    res = req.request('GET', url)
    soup = BeautifulSoup(res.data, 'html.parser')
    contents = soup.find_all(class_ = 'table4')
    return contents

In [695]:
womens_100m_16 = olympic_query('100m-women')
womens_100m_16 = fn.athens_scraper(womens_100m_16)
womens_100m_16 = fn.content_cleaner(womens_100m_16)
womens_100m_16.columns = ['rank', 'name', 'result']
womens_100m_16.insert(3, "event_x", "Athletics Women's 100 metres")
womens_100m_16.name = [x.strip() for x in womens_100m_16.name]
womens_100m_16 = womens_100m_16.replace('Marie-Josee Ta Lou', 'Marie-Jose Lou')

womens_100m_16

,rank,name,result,event_x
0,1.,Elaine Thompson,10.71,Athletics Women's 100 metres
1,2.,Tori Bowie,10.83,Athletics Women's 100 metres
2,3.,Shelly-Ann Fraser-Pryce,10.86,Athletics Women's 100 metres
3,4.,Marie-Jose Lou,10.86,Athletics Women's 100 metres
4,5.,Dafne Schippers,10.90,Athletics Women's 100 metres
5,6.,Michelle-Lee Ahye,10.92,Athletics Women's 100 metres
6,7.,English Gardner,10.94,Athletics Women's 100 metres
7,8.,Christania Williams,11.80,Athletics Women's 100 metres


In [696]:
df_16 = pd.merge(df_16, womens_100m_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_100m_womens'))
df_16.head(30)

,id,name,sex,age,height,weight,team,noc,year,season,city,sport,event_x,medal,event_y,substance,flagged,result,result_100m_womens
0,55,Antonio Beci,M,26.0,170.0,65.0,Spain,ESP,2016,Summer,Rio de Janeiro,Athletics,"Athletics Men's 5,000 metres",NaN,NaN,NaN,NaN,NaN,NaN
1,110,Abubakar Abbas,M,20.0,175.0,66.0,Bahrain,BRN,2016,Summer,Rio de Janeiro,Athletics,Athletics Men's 400 metres,NaN,NaN,NaN,NaN,NaN,NaN
2,251,Bashir Abdi,M,27.0,176.0,56.0,Belgium,BEL,2016,Summer,Rio de Janeiro,Athletics,"Athletics Men's 5,000 metres",NaN,NaN,NaN,NaN,NaN,NaN
3,251,Bashir Abdi,M,27.0,176.0,56.0,Belgium,BEL,2016,Summer,Rio de Janeiro,Athletics,"Athletics Men's 10,000 metres",NaN,NaN,NaN,NaN,NaN,NaN
4,273,Maizurah Rahim,F,17.0,147.0,50.0,Brunei,BRU,2016,Summer,Rio de Janeiro,Athletics,Athletics Women's 200 metres,NaN,NaN,NaN,NaN,NaN,NaN
5,402,Arthur Abele,M,30.0,184.0,85.0,Germany,GER,2016,Summer,Rio de Janeiro,Athletics,Athletics Men's Decathlon,NaN,NaN,NaN,NaN,NaN,NaN
6,412,Tesfaye Dibaba,M,24.0,192.0,68.0,Ethiopia,ETH,2016,Summer,Rio de Janeiro,Athletics,Athletics Men's Marathon,NaN,NaN,NaN,NaN,NaN,NaN
7,511,Tadesse Abraham,M,33.0,178.0,61.0,Switzerland,SUI,2016,Summer,Rio de Janeiro,Athletics,Athletics Men's Marathon,NaN,NaN,NaN,NaN,NaN,NaN
8,533,Yelena Abramchuk,F,28.0,182.0,95.0,Belarus,BLR,2016,Summer,Rio de Janeiro,Athletics,Athletics Women's Shot Put,NaN,NaN,NaN,NaN,NaN,NaN
9,551,Aliyah Abrams,F,19.0,163.0,53.0,Guyana,GUY,2016,Summer,Rio de Janeiro,Athletics,Athletics Women's 400 metres,NaN,NaN,NaN,NaN,NaN,NaN


### Next, I will join the 200m dash results

In [697]:
mens_200m_16 = olympic_query('200m-men')
mens_200m_16 = fn.athens_scraper(mens_200m_16)
mens_200m_16 = fn.content_cleaner(mens_200m_16)
mens_200m_16.columns = ['rank', 'name', 'result']
mens_200m_16.insert(3, "event_x", "Athletics Men's 200 metres")
mens_200m_16 = format_name(mens_200m_16)
mens_200m_16.name = mens_200m_16.name.replace('Lashawn Merrit', 'LaShawn Merrit')
mens_200m_16.name = mens_200m_16.name.replace('Alonso Edward', 'Alonso Henry')
mens_200m_16.name = mens_200m_16.name.replace('Christophe Lemaitre', 'Christophe Lematre')

mens_200m_16

,rank,name,result,event_x
0,1.,Usain Bolt,19.78,Athletics Men's 200 metres
1,2.,Andre Grasse,20.02,Athletics Men's 200 metres
2,3.,Christophe Lematre,20.12,Athletics Men's 200 metres
3,4.,Adam Gemili,20.12,Athletics Men's 200 metres
4,5.,Churandy Martina,20.13,Athletics Men's 200 metres
5,6.,Lashawn Merritt,20.19,Athletics Men's 200 metres
6,7.,Alonso Henry,20.23,Athletics Men's 200 metres
7,8.,Ramil Guliyev,20.43,Athletics Men's 200 metres


In [698]:
df_16 = pd.merge(df_16, mens_200m_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_200m_mens'))

In [699]:
df_16.head()

,id,name,sex,age,height,weight,team,noc,year,season,city,sport,event_x,medal,event_y,substance,flagged,result,result_100m_womens,result_200m_mens
0,55,Antonio Beci,M,26.0,170.0,65.0,Spain,ESP,2016,Summer,Rio de Janeiro,Athletics,"Athletics Men's 5,000 metres",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110,Abubakar Abbas,M,20.0,175.0,66.0,Bahrain,BRN,2016,Summer,Rio de Janeiro,Athletics,Athletics Men's 400 metres,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,251,Bashir Abdi,M,27.0,176.0,56.0,Belgium,BEL,2016,Summer,Rio de Janeiro,Athletics,"Athletics Men's 5,000 metres",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,251,Bashir Abdi,M,27.0,176.0,56.0,Belgium,BEL,2016,Summer,Rio de Janeiro,Athletics,"Athletics Men's 10,000 metres",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,273,Maizurah Rahim,F,17.0,147.0,50.0,Brunei,BRU,2016,Summer,Rio de Janeiro,Athletics,Athletics Women's 200 metres,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [700]:
names = []
for x in womens_200m_16.name:
    split = x.split()
    x = split[0] + ' ' + split[-1]
    names.append(x)


womens_200m_16['name'] = [x for x in names]

In [701]:
womens_200m_16 = olympic_query('200m-women')
womens_200m_16 = fn.athens_scraper(womens_200m_16)
womens_200m_16 = fn.content_cleaner(womens_200m_16)
womens_200m_16.columns = ['rank', 'name', 'result']
womens_200m_16.insert(3, "event_x", "Athletics Women's 200 metres")
womens_200m_16.name = [x.strip() for x in womens_200m_16.name]
womens_200m_16 = womens_200m_16.replace('Marie-Josee Ta Lou', 'Marie-Jose Lou')
womens_200m_16 = womens_200m_16.replace('Dina Asher-Smith', 'Geraldina Asher-Smith')
womens_200m_16

,rank,name,result,event_x
0,1.,Elaine Thompson,21.78,Athletics Women's 200 metres
1,2.,Dafne Schippers,21.88,Athletics Women's 200 metres
2,3.,Tori Bowie,22.15,Athletics Women's 200 metres
3,4.,Marie-Jose Lou,22.21,Athletics Women's 200 metres
4,5.,Geraldina Asher-Smith,22.31,Athletics Women's 200 metres
5,6.,Michelle-Lee Ahye,22.34,Athletics Women's 200 metres
6,7.,Deajah Stevens,22.65,Athletics Women's 200 metres
7,8.,Ivet Lalova-Collio,22.69,Athletics Women's 200 metres


In [702]:
df_16 = pd.merge(df_16, womens_200m_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_200m_womens'))
df_16.head()

,id,name,sex,age,height,weight,team,noc,year,season,city,sport,event_x,medal,event_y,substance,flagged,result,result_100m_womens,result_200m_mens,result_200m_womens
0,55,Antonio Beci,M,26.0,170.0,65.0,Spain,ESP,2016,Summer,Rio de Janeiro,Athletics,"Athletics Men's 5,000 metres",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110,Abubakar Abbas,M,20.0,175.0,66.0,Bahrain,BRN,2016,Summer,Rio de Janeiro,Athletics,Athletics Men's 400 metres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,251,Bashir Abdi,M,27.0,176.0,56.0,Belgium,BEL,2016,Summer,Rio de Janeiro,Athletics,"Athletics Men's 5,000 metres",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,251,Bashir Abdi,M,27.0,176.0,56.0,Belgium,BEL,2016,Summer,Rio de Janeiro,Athletics,"Athletics Men's 10,000 metres",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,273,Maizurah Rahim,F,17.0,147.0,50.0,Brunei,BRU,2016,Summer,Rio de Janeiro,Athletics,Athletics Women's 200 metres,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [703]:
def content_cleaner(df):
    df = df.dropna(axis=0)
    df['rank'] = [x.strip('\n') for x in df['rank']]
    df['rank'] = [x.replace('G', '1.') for x in df['rank']]
    df['rank'] = [x.replace('S', '2.') for x in df['rank']]
    df['rank'] = [x.replace('B', '3.') for x in df['rank']]
    
    df['name'] = [x.strip('\n\n\n\n\n\n\n\n\n\n\n\n') for x in df['name']]
    df['name'] = [x.replace('\n', ' ') for x in df['name']]
    df['name'] = [x[:-3] for x in df['name']]
    df['name'] = [x.title() for x in df['name']]   
    
    df['result'] = [x[:-4] for x in df['result']]
    df['result'] = [x.split('\r\n') for x in df['result']]
    df['result'] = [x[1] for x in df['result']]
    df['result'] = [x.replace('h', ':') for x in df['result']]
    
    return df


In [704]:
mens_110h_16 = olympic_query('110m-hurdles-men')
mens_110h_16 = fn.athens_scraper(mens_110h_16)
mens_110h_16 = mens_110h_16.drop(7)
mens_110h_16 = content_cleaner(mens_110h_16)
mens_110h_16.columns = ['rank', 'name', 'result']
mens_110h_16.insert(3, "event_x", "Athletics Men's 110 metres Hurdles")
mens_110h_16 = format_name(mens_110h_16)
mens_110h_16 = mens_110h_16.replace('Omar Mcleod', 'Omar McLeod')
mens_110h_16 = mens_110h_16.replace('Orlando Ortega', 'Orlando Alejo')

mens_110h_16

,rank,name,result,event_x
0,1.,Omar McLeod,13.05,Athletics Men's 110 metres Hurdles
1,2.,Orlando Alejo,13.17,Athletics Men's 110 metres Hurdles
2,3.,Dimitri Bascou,13.24,Athletics Men's 110 metres Hurdles
3,4.,Pascal Martinot-Lagarde,13.29,Athletics Men's 110 metres Hurdles
4,5.,Devon Allen,13.31,Athletics Men's 110 metres Hurdles
5,6.,Johnathan Cabral,13.40,Athletics Men's 110 metres Hurdles
6,7.,Milan Trajkovic,13.41,Athletics Men's 110 metres Hurdles


In [705]:
df_16 = pd.merge(df_16, mens_110h_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_110h_mens'))

In [706]:
df_16.result.value_counts()

9.89     1
10.06    1
9.93     1
9.81     1
9.94     1
9.91     1
10.04    1
Name: result, dtype: int64

In [707]:
df_16.result_110h_mens.value_counts()

13.40    1
13.17    1
13.41    1
13.31    1
13.24    1
13.29    1
13.05    1
Name: result_110h_mens, dtype: int64

In [708]:
womens_110h_16 = olympic_query('100m-hurdles-women')
womens_110h_16 = fn.athens_scraper(womens_110h_16)
womens_110h_16 = content_cleaner(womens_110h_16)
womens_110h_16.columns = ['rank', 'name', 'result']
womens_110h_16.insert(3, "event_x", "Athletics Women's 100 metres Hurdles")
womens_110h_16 = format_name(womens_110h_16)
#womens_110h_16 = womens_110h_16.replace('Omar Mcleod', 'Omar McLeod')
#womens_110h_16 = womens_110h_16.replace('Orlando Ortega', 'Orlando Alejo')

womens_110h_16

,rank,name,result,event_x
0,1.,Brianna Rollins,12.48,Athletics Women's 100 metres Hurdles
1,2.,Nia Ali,12.59,Athletics Women's 100 metres Hurdles
2,3.,Kristi Castlin,12.61,Athletics Women's 100 metres Hurdles
3,4.,Cindy Ofili,12.63,Athletics Women's 100 metres Hurdles
4,5.,Cindy Roleder,12.74,Athletics Women's 100 metres Hurdles
5,6.,Pedrya Seymour,12.76,Athletics Women's 100 metres Hurdles
6,7.,Tiffany Porter,12.76,Athletics Women's 100 metres Hurdles
7,8.,Phylicia George,12.89,Athletics Women's 100 metres Hurdles


In [709]:
df_16 = pd.merge(df_16, womens_110h_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_100h_womens'))
df_16.result_100h_womens.value_counts()

12.76    2
12.63    1
12.59    1
12.48    1
12.61    1
12.89    1
12.74    1
Name: result_100h_womens, dtype: int64

In [710]:
df.columns

Index(['id', 'name', 'sex', 'age', 'height', 'weight', 'team', 'noc', 'year', 'season', 'city', 'sport', 'event_x', 'medal', 'event_y', 'substance', 'flagged'], dtype='object')

In [711]:
pd.pivot_table(df, index=['event_x'], values='flagged', aggfunc='count')

,flagged
event_x,
"Athletics Men's 1,500 metres",0
"Athletics Men's 10,000 metres",16
Athletics Men's 100 metres,176
Athletics Men's 110 metres Hurdles,53
Athletics Men's 20 kilometres Walk,59
Athletics Men's 200 metres,190
"Athletics Men's 3,000 metres Steeplechase",125
Athletics Men's 4 x 100 metres Relay,97
Athletics Men's 4 x 400 metres Relay,57


### 3000m Steeplechase

In [712]:
mens_3000steep_16 = olympic_query('3000m-steeplechase-men')
mens_3000steep_16 = fn.athens_scraper(mens_3000steep_16)
mens_3000steep_16 = mens_3000steep_16.drop([11,12,13,14])
mens_3000steep_16 = content_cleaner(mens_3000steep_16)
mens_3000steep_16.columns = ['rank', 'name', 'result']
mens_3000steep_16.insert(3, "event_x", "Athletics Men's 3,000 metres Steeplechase")
mens_3000steep_16 = format_name(mens_3000steep_16)
mens_3000steep_16 = mens_3000steep_16.replace('Mahiedine Mekhissi', 'Mahiedine Mekhissi-Benabbad')
mens_3000steep_16 = mens_3000steep_16.replace('Soufiane Elbakkali', 'Soufiane El-Bakkali')

mens_3000steep_16

,rank,name,result,event_x
0,1.,Conseslus Kipruto,8:03.28,"Athletics Men's 3,000 metres Steeplechase"
1,2.,Evan Jager,8:04.28,"Athletics Men's 3,000 metres Steeplechase"
2,3.,Mahiedine Mekhissi-Benabbad,8:11.52,"Athletics Men's 3,000 metres Steeplechase"
3,4.,Soufiane El-Bakkali,8:14.35,"Athletics Men's 3,000 metres Steeplechase"
4,5.,Yoann Kowal,8:16.75,"Athletics Men's 3,000 metres Steeplechase"
5,6.,Brimin Kipruto,8:18.79,"Athletics Men's 3,000 metres Steeplechase"
6,7.,Hillary Bor,8:22.74,"Athletics Men's 3,000 metres Steeplechase"
7,8.,Donald Cabral,8:25.81,"Athletics Men's 3,000 metres Steeplechase"
8,9.,Altobeli Silva,8:26.30,"Athletics Men's 3,000 metres Steeplechase"
9,10.,Matthew Hughes,8:36.83,"Athletics Men's 3,000 metres Steeplechase"


In [713]:
df_16 = pd.merge(df_16, mens_3000steep_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_3000steep_mens'))

In [714]:
df_16.result_3000steep_mens.value_counts()

8:16.75    1
8:04.28    1
8:26.30    1
8:14.35    1
8:22.74    1
8:11.52    1
8:18.79    1
8:40.68    1
8:03.28    1
8:25.81    1
8:36.83    1
Name: result_3000steep_mens, dtype: int64

In [715]:
womens_3000steep_16 = olympic_query('3000m-steeplechase-women')
womens_3000steep_16 = fn.athens_scraper(womens_3000steep_16)
womens_3000steep_16 = womens_3000steep_16.drop([11,12,13,14])
womens_3000steep_16 = content_cleaner(womens_3000steep_16)
womens_3000steep_16.columns = ['rank', 'name', 'result']
womens_3000steep_16.insert(3, "event_x", "Athletics Men's Marathon	")
womens_3000steep_16 = format_name(womens_3000steep_16)
# mens_3000steep_16 = mens_3000steep_16.replace('Mahiedine Mekhissi', 'Mahiedine Mekhissi-Benabbad')
# mens_3000steep_16 = mens_3000steep_16.replace('Soufiane Elbakkali', 'Soufiane El-Bakkali')

womens_3000steep_16

,rank,name,result,event_x
0,1.,Ruth Jebet,8:59.75,Athletics Men's Marathon\t
1,2.,Hyvin Jepkemoi,9:07.12,Athletics Men's Marathon\t
2,3.,Emma Coburn,9:07.63,Athletics Men's Marathon\t
3,4.,Beatrice Chepkoech,9:16.05,Athletics Men's Marathon\t
4,5.,Sofia Assefa,9:17.15,Athletics Men's Marathon\t
5,6.,Gesa Krause,9:18.41,Athletics Men's Marathon\t
6,7.,Madeline Hills,9:20.38,Athletics Men's Marathon\t
7,8.,Colleen Quigley,9:21.10,Athletics Men's Marathon\t
8,9.,Genevieve Lacaze,9:21.21,Athletics Men's Marathon\t
9,10.,Lalita Babar,9:22.74,Athletics Men's Marathon\t


In [716]:
df_16 = pd.merge(df_16, womens_3000steep_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_3000steep_womens'))
df_16.result_3000steep_womens.value_counts()

Series([], Name: result_3000steep_womens, dtype: int64)

### Marathon

In [717]:
mens_marathon_16 = olympic_query('marathon-men')
mens_marathon_16 = fn.athens_scraper(mens_marathon_16)
mens_marathon_16 = mens_marathon_16.drop([140,141,142,143,144,145,146,147,148,149,150,151,152,153,154])
mens_marathon_16 = content_cleaner(mens_marathon_16)
mens_marathon_16.columns = ['rank', 'name', 'result']
mens_marathon_16.insert(3, "event_x", "Athletics Men's Marathon")
mens_marathon_16 = format_name(mens_marathon_16)
# mens_marathon_16 = mens_marathon_16.replace('Mahiedine Mekhissi', 'Mahiedine Mekhissi-Benabbad')
# mens_marathon_16 = mens_marathon_16.replace('Soufiane Elbakkali', 'Soufiane El-Bakkali')

mens_marathon_16

,rank,name,result,event_x
0,1.,Eliud Rotich,2:08:44,Athletics Men's Marathon
1,2.,Feyisa Lilesa,2:09:54,Athletics Men's Marathon
2,3.,Galen Rupp,2:10:05,Athletics Men's Marathon
3,4.,Ghirmay Ghebreslassie,2:11:04,Athletics Men's Marathon
4,5.,Alphonce Simbu,2:11:15,Athletics Men's Marathon
5,6.,Jared Ward,2:11:30,Athletics Men's Marathon
6,7.,Tadesse Abraham,2:11:42,Athletics Men's Marathon
7,8.,Munyo Mutai,2:11:49,Athletics Men's Marathon
8,9.,Callum Hawkins,2:11:52,Athletics Men's Marathon
9,10.,Eric Gillis,2:12:29,Athletics Men's Marathon


In [718]:
marathon_names = [x for x in mens_marathon_16.name]

In [719]:
df_16 = pd.merge(df_16, mens_marathon_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_marathon_mens'))
df_16.result_marathon_mens.value_counts()

2:17:06    2
2:18:56    2
2:18:44    1
2:31:27    1
2:26:16    1
2:20:24    1
2:17:08    1
2:20:01    1
2:31:54    1
2:16:46    1
2:17:59    1
2:25:33    1
2:40:05    1
2:27:01    1
2:30:52    1
2:13:56    1
2:22:19    1
2:31:07    1
2:17:30    1
2:17:27    1
2:39:25    1
2:14:53    1
2:13:57    1
2:13:01    1
2:30:30    1
2:15:31    1
2:19:26    1
2:20:47    1
2:20:06    1
2:11:49    1
2:26:47    1
2:14:58    1
2:11:42    1
2:11:15    1
2:17:49    1
2:16:12    1
2:09:54    1
2:25:57    1
2:21:17    1
2:18:58    1
2:13:04    1
2:14:17    1
2:37:46    1
2:10:05    1
2:11:04    1
2:12:29    1
2:22:05    1
2:28:54    1
2:17:48    1
2:18:19    1
2:16:24    1
2:21:00    1
2:45:55    1
2:25:11    1
2:15:36    1
2:22:09    1
2:18:57    1
2:18:36    1
2:46:18    1
2:18:40    1
2:30:45    1
2:19:50    1
2:15:26    1
2:25:13    1
2:11:52    1
2:15:24    1
2:22:45    1
2:18:06    1
2:29:24    1
2:13:32    1
2:24:58    1
2:38:29    1
2:28:21    1
2:24:08    1
2:11:30    1
Name: result_marathon_men

In [720]:
womens_marathon_16 = olympic_query('marathon-women')
womens_marathon_16 = fn.athens_scraper(womens_marathon_16)
womens_marathon_16 = womens_marathon_16.drop([133,134,135,136,137,138,139,140,141,142,143,
                                              144,145,146,147,148,149,150,151,152,153,154,
                                              155,156])
womens_marathon_16 = content_cleaner(womens_marathon_16)
womens_marathon_16.columns = ['rank', 'name', 'result']
womens_marathon_16.insert(3, "event_x", "Athletics Women's Marathon")
womens_marathon_16 = format_name(womens_marathon_16)
# mens_marathon_16 = mens_marathon_16.replace('Mahiedine Mekhissi', 'Mahiedine Mekhissi-Benabbad')
# mens_marathon_16 = mens_marathon_16.replace('Soufiane Elbakkali', 'Soufiane El-Bakkali')

womens_marathon_16

,rank,name,result,event_x
0,1.,Jemima Sumgong,2:24:04,Athletics Women's Marathon
1,2.,Eunice Kirwa,2:24:13,Athletics Women's Marathon
2,3.,Mare Dibaba,2:24:30,Athletics Women's Marathon
3,4.,Tirfi Tsegaye,2:24:47,Athletics Women's Marathon
4,5.,Volha Mazuronak,2:24:48,Athletics Women's Marathon
5,6.,Shalane Flanagan,2:25:26,Athletics Women's Marathon
6,7.,Desiree Linden,2:26:08,Athletics Women's Marathon
7,8.,Rose Chelimo,2:27:36,Athletics Women's Marathon
8,9.,Amy Cragg,2:28:25,Athletics Women's Marathon
9,10.,Hye-Song Kim,2:28:36,Athletics Women's Marathon


In [721]:
df_16 = pd.merge(df_16, womens_marathon_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_marathon_womens'))
df_16.result_marathon_womens.value_counts()

2:47:27    2
2:46:05    1
2:39:55    1
2:27:36    1
2:53:57    1
2:48:34    1
2:42:39    1
2:47:53    1
2:51:08    1
3:11:31    1
3:16:11    1
2:34:11    1
2:46:08    1
2:36:14    1
2:31:44    1
2:44:18    1
2:29:44    1
2:33:29    1
2:30:53    1
2:41:02    1
2:43:22    1
3:00:31    1
2:25:26    1
2:37:39    1
2:44:41    1
2:45:32    1
2:52:31    1
2:40:49    1
3:09:15    1
2:37:37    1
2:31:12    1
2:24:13    1
2:48:29    1
2:36:32    1
2:59:29    1
2:42:03    1
2:24:04    1
2:41:52    1
3:20:20    1
2:41:37    1
2:31:41    1
2:34:36    1
2:43:52    1
2:45:08    1
2:45:52    1
2:39:49    1
2:48:24    1
2:50:51    1
2:51:06    1
2:34:57    1
2:48:52    1
2:41:46    1
3:05:57    1
2:45:33    1
2:54:38    1
2:34:41    1
2:33:08    1
2:29:53    1
3:00:03    1
2:37:23    1
2:49:07    1
2:44:20    1
2:39:59    1
3:02:27    1
2:47:32    1
2:34:27    1
Name: result_marathon_womens, dtype: int64

### 20km Walk

In [722]:
mens_20km_16 = olympic_query('20km-walk-men')
mens_20km_16 = fn.athens_scraper(mens_20km_16)
mens_20km_16 = mens_20km_16.drop([63,64,65,66,67,68,69,70,71,72,73])
mens_20km_16 = content_cleaner(mens_20km_16)
mens_20km_16.columns = ['rank', 'name', 'result']
mens_20km_16.insert(3, "event_x", "Athletics Men's 20 kilometres Walk")
mens_20km_16 = format_name(mens_20km_16)
# mens_marathon_16 = mens_marathon_16.replace('Mahiedine Mekhissi', 'Mahiedine Mekhissi-Benabbad')
# mens_marathon_16 = mens_marathon_16.replace('Soufiane Elbakkali', 'Soufiane El-Bakkali')

mens_20km_16

,rank,name,result,event_x
0,1.,Zhen Wang,1:19:14,Athletics Men's 20 kilometres Walk
1,2.,Zelin Cai,1:19:26,Athletics Men's 20 kilometres Walk
2,3.,Dane Bird-Smith,1:19:37,Athletics Men's 20 kilometres Walk
3,4.,Caio Bonfim,1:19:42,Athletics Men's 20 kilometres Walk
4,5.,Christopher Linke,1:20:00,Athletics Men's 20 kilometres Walk
5,6.,Tom Bosworth,1:20:13,Athletics Men's 20 kilometres Walk
6,7.,Daisuke Matsunaga,1:20:22,Athletics Men's 20 kilometres Walk
7,8.,Matteo Giupponi,1:20:27,Athletics Men's 20 kilometres Walk
8,9.,Manuel Soto,1:20:36,Athletics Men's 20 kilometres Walk
9,10.,Evan Dunfee,1:20:49,Athletics Men's 20 kilometres Walk


In [723]:
df_16 = pd.merge(df_16, mens_20km_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_20km_mens'))
df_16.result_20km_mens.value_counts()

1:19:37    1
1:21:52    1
1:29:20    1
1:25:07    1
1:23:46    1
1:20:00    1
1:22:28    1
1:22:03    1
1:19:42    1
1:25:25    1
1:24:40    1
1:33:58    1
1:20:27    1
1:20:49    1
1:20:22    1
1:21:40    1
1:22:53    1
1:25:29    1
1:24:59    1
1:23:30    1
1:26:06    1
1:21:44    1
Name: result_20km_mens, dtype: int64

In [724]:
len(df_16.result_20km_mens.value_counts())

22

In [725]:
womens_20km_16 = olympic_query('20km-race-walk-women')
womens_20km_16 = fn.athens_scraper(womens_20km_16)
womens_20km_16 = womens_20km_16.drop([63,64,65,66,67,68,69,70,71,72,73])
womens_20km_16 = content_cleaner(womens_20km_16)
womens_20km_16.columns = ['rank', 'name', 'result']
womens_20km_16.insert(3, "event_x", "Athletics Women's 20 kilometres Walk")
womens_20km_16 = format_name(womens_20km_16)
# mens_marathon_16 = mens_marathon_16.replace('Mahiedine Mekhissi', 'Mahiedine Mekhissi-Benabbad')
# mens_marathon_16 = mens_marathon_16.replace('Soufiane Elbakkali', 'Soufiane El-Bakkali')

womens_20km_16

,rank,name,result,event_x
0,1.,Hong Liu,1:28:35,Athletics Women's 20 kilometres Walk
1,2.,Maria Gonzalez,1:28:37,Athletics Women's 20 kilometres Walk
2,3.,Xiuzhi Lu,1:28:42,Athletics Women's 20 kilometres Walk
3,4.,Antonella Palmisano,1:29:03,Athletics Women's 20 kilometres Walk
4,5.,Shijie Qieyang,1:29:04,Athletics Women's 20 kilometres Walk
5,6.,Ana Cabecinha,1:29:23,Athletics Women's 20 kilometres Walk
6,7.,Erica Sena,1:29:29,Athletics Women's 20 kilometres Walk
7,8.,Beatriz Pascual,1:30:24,Athletics Women's 20 kilometres Walk
8,9.,Regan Lamble,1:30:28,Athletics Women's 20 kilometres Walk
9,10.,Anezka Drahotova,1:30:43,Athletics Women's 20 kilometres Walk


In [726]:
df_16 = pd.merge(df_16, womens_20km_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_20km_womens'))
df_16.result_20km_womens.value_counts()

1:38:35    1
1:29:03    1
1:37:49    1
1:35:01    1
1:29:23    1
1:38:20    1
1:30:28    1
1:33:36    1
1:32:42    1
1:35:48    1
1:38:16    1
1:31:04    1
1:34:22    1
1:45:06    1
1:37:08    1
1:36:13    1
1:38:42    1
1:40:15    1
1:35:55    1
1:33:15    1
1:42:57    1
1:40:33    1
Name: result_20km_womens, dtype: int64

In [727]:
len(df_16.result_20km_womens.value_counts())

22

### 400m

In [729]:
mens_400m_16 = olympic_query('400m-men')
mens_400m_16 = fn.athens_scraper(mens_400m_16)
mens_400m_16 = content_cleaner(mens_400m_16)
mens_400m_16.columns = ['rank', 'name', 'result']
mens_400m_16.insert(3, "event_x", "Athletics Men's 400 metres")
mens_400m_16 = format_name(mens_400m_16)

mens_400m_16

,rank,name,result,event_x
0,1.,Wayde Niekerk,43.03,Athletics Men's 400 metres
1,2.,Kirani James,43.76,Athletics Men's 400 metres
2,3.,Lashawn Merritt,43.85,Athletics Men's 400 metres
3,4.,Machel Cedenio,44.01,Athletics Men's 400 metres
4,5.,Karabo Sibanda,44.25,Athletics Men's 400 metres
5,6.,Ali Khamis,44.36,Athletics Men's 400 metres
6,7.,Bralon Taplin,44.45,Athletics Men's 400 metres
7,8.,Matthew Hudson-Smith,44.61,Athletics Men's 400 metres


In [730]:
df_16 = pd.merge(df_16, mens_400m_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_400m_mens'))
df_16.result_400m_mens.value_counts()

44.36    1
44.01    1
44.25    1
43.03    1
44.61    1
44.45    1
43.76    1
Name: result_400m_mens, dtype: int64

In [731]:
womens_400m_16 = olympic_query('400m-women')
womens_400m_16 = fn.athens_scraper(womens_400m_16)
womens_400m_16 = content_cleaner(womens_400m_16)
womens_400m_16.columns = ['rank', 'name', 'result']
womens_400m_16.insert(3, "event_x", "Athletics Women's 400 metres")
womens_400m_16 = format_name(womens_400m_16)

womens_400m_16

,rank,name,result,event_x
0,1.,Shaunae Miller,49.44,Athletics Women's 400 metres
1,2.,Allyson Felix,49.51,Athletics Women's 400 metres
2,3.,Shericka Jackson,49.85,Athletics Women's 400 metres
3,4.,Natasha Hastings,50.34,Athletics Women's 400 metres
4,5.,Phyllis Francis,50.41,Athletics Women's 400 metres
5,6.,Stephenie Mcpherson,50.97,Athletics Women's 400 metres
6,7.,Olha Zemlyak,51.24,Athletics Women's 400 metres
7,8.,Libania Martinez,51.25,Athletics Women's 400 metres


In [732]:
df_16 = pd.merge(df_16, womens_400m_16[['name', 'result', 'event_x']], how='left', on=['name', 'event_x'], suffixes=(None, '_400m_womens'))
df_16.result_400m_womens.value_counts()

49.44    1
50.41    1
50.34    1
49.51    1
49.85    1
Name: result_400m_womens, dtype: int64